In [1]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf                
from tqdm import tqdm

import numpy as np
import matplotlib.pylab as plt

import itertools
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import keras
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
%matplotlib inline

In [2]:
class_names = ['Bengin cases', 'Malignant cases', 'Normal cases']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)

IMAGE_SIZE = (224, 224)

In [3]:
def load_data():
  
    datasets = ['Train', 'Test']
    output = []
    
    
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
            
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE) 
                
                
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

In [4]:
(train_images, train_labels), (test_images, test_labels) = load_data()

  0%|                                                                                          | 0/391 [00:00<?, ?it/s]

Loading Train


 58%|██████████████████████████████████████████████▌                                 | 99/170 [00:00<00:00, 986.91it/s]

Loading Test


100%|███████████████████████████████████████████████████████████████████████████████| 126/126 [00:00<00:00, 742.36it/s]


In [5]:
train_images, train_labels = shuffle(train_images, train_labels, random_state=25)

In [6]:
train_images = train_images / 255.0 
test_images = test_images / 255.0

In [7]:
from keras.applications.vgg16 import VGG16
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [8]:
for layer in VGG_model.layers:
    layer.trainable = False
    

In [9]:
feature_extractor=VGG_model.predict(train_images)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

In [11]:
X_test_feature = VGG_model.predict(test_images)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

In [13]:
from sklearn import tree
classifier = tree.DecisionTreeClassifier()
classifier.fit(features, train_labels)

DecisionTreeClassifier()

In [14]:
y_pred = classifier.predict(X_test_features)

In [15]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(test_labels, y_pred))
print(classification_report(test_labels, y_pred))

[[ 10   5  11]
 [ 16 119  35]
 [ 41  17  68]]
              precision    recall  f1-score   support

           0       0.15      0.38      0.22        26
           1       0.84      0.70      0.77       170
           2       0.60      0.54      0.57       126

    accuracy                           0.61       322
   macro avg       0.53      0.54      0.52       322
weighted avg       0.69      0.61      0.64       322



In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(test_labels, y_pred)

0.6118012422360248